In [175]:
import requests
import json
# Date format MM/DD/YYYY
# Site DK=20 FD=2
DATE = "08/05/2019"
SITE = "20"
# Vars -- DATE, SITE
# This one can be used to get the player salaries
SLATES = "https://resultsdb-api.rotogrinders.com/api/slates?start=DATE&site=SITE"
# Vars -- SLATEID
CONTEST_OWNERSHIP = "https://resultsdb-api.rotogrinders.com/api/contest-ownership?_slateId=SLATEID"
# Vars -- CONTESTID, INDEXNUM
CONTEST_LINEUPS = "https://resultsdb-api.rotogrinders.com/api/entries?_contestId=CONTESTID&sortBy=points&order=desc&index=INDEXNUM&maxFinish=58750&players=&users=false&username=&isLive=false&minPoints=&maxSalaryUsed=&incomplete=false&positionsRemaining=&requiredStartingPositions="
# Vars SLATEID
CONTEST_SLATE = "https://resultsdb-api.rotogrinders.com/api/contests?slates=SLATEID&lean=true"
# Vars -- SLATEID (get summary)
SUMMARY = ""

In [98]:
"""
    pulls the different slates based on the date and the site
    date- date of the event 
    site- defaults to Draftkings=20, Fanduel=2
    sport- filters the slate by sport, defaults to None
    slate_type- fitlers on the type of contest, Classic by default
"""
sport_mapping = {"football": 1, "baseball": 2, "basketball": 3}
def get_slates(date, site="20", sport=None, slate_type="Classic"):
    slate = SLATES.replace("DATE", date).replace("SITE", site)
    slate_data = requests.get(slate).json()
    if sport is not None:
        slate_data = [slate for slate in slate_data if slate['sport'] == sport_mapping[sport]]
    if slate_type is not None:
        slate_data = [slate for slate in slate_data if slate['slateTypeName'] == slate_type]
    return slate_data

In [99]:
"""
    Fetches the ownerships for a particular slate. Returns the data as a dictionary of players 
    with individual contest ownership after
    slate_id - id of the slate found with the get_slates function
    returns daily_ownerships dict
"""
def get_contest_ownerships(slate_id):
    contest_ownership_url = CONTEST_OWNERSHIP.replace("SLATEID", slate_id)
    daily_ownerships = requests.get(contest_ownership_url).json()
    return daily_ownerships

In [100]:
"""
    Fetches the contests from the slate on that day.
    slate_id -- id of the slate to find contests for
    returns contests in that slate
"""
def get_contests_from_slate(slate_id):
    contests_in_slate_url = CONTEST_SLATE.replace("SLATEID", slate_id)
    contests_in_slate = requests.get(contests_in_slate_url).json()
    return contests_in_slate

In [101]:
"""
    Fetches the lineups 25 at a time based on the contest_id and index passed
    contest_id- id of the contest to fetch (found in the summary call)
    index- index of the lineups to fetch (fetches number 25*(index) - 25*(index+1))
    returns lineups
"""
def get_contest_lineups(contest_id, index=0):
    contest_lineup_url = CONTEST_LINEUPS.replace("CONTESTID", contest_id)
    daily_lineups = requests.get(contest_lineup_url).json()
    return daily_lineups

In [176]:
# Random snippets
daily_slate = get_slates(DATE, SITE, sport="baseball")
SLATEID = daily_slate[1]['_id']
daily_contest_lineups = get_contest_lineups(CONTEST_ID)
contests_in_slate = get_contests_from_slate(SLATEID)
contests_in_slate[0].keys()
CONTEST_ID = "5d7f7a67d0585a501d5c53be"
daily_ownerships = get_contest_ownerships(SLATEID)
daily_ownerships.keys()
contests_in_slate[0]
daily_ownerships['Anthony Rizzo']
daily_ownerships.keys()
# daily slates have line data in them
daily_slate[1]

{'_id': '5d48b6c65c98993fbdee1aad',
 'isSalary': True,
 'slateTypeName': 'Classic',
 'gameCount': 12,
 'siteSlateId': '28763',
 'end': '2019-08-06T02:10:00.000Z',
 'start': '2019-08-05T23:05:00.000Z',
 'sport': 2,
 'siteId': 20,
 'slateType': 2,
 'gameTypeId': 2,
 'updatedAt': '2019-08-06T12:10:06.515Z',
 '__v': 41,
 'startingPositions': ['P',
  'P',
  'C',
  '1B',
  '2B',
  '3B',
  'SS',
  'OF',
  'OF',
  'OF'],
 'complete': True,
 'optimalLineup': [{'name': 'Luis Castillo',
   'id': '13077587',
   'salary': 10100,
   'fpts': 39.35,
   'per_point': 256.67,
   'position': 'P',
   'ownership': '16.67%'},
  {'name': 'Rick Porcello',
   'id': '13077601',
   'salary': 6400,
   'fpts': 21.9,
   'per_point': 292.24,
   'position': 'P',
   'ownership': '44.44%'},
  {'name': 'James McCann',
   'id': '13077937',
   'salary': 3400,
   'fpts': 16,
   'per_point': 212.5,
   'position': 'C',
   'ownership': '5.56%'},
  {'name': 'Joc Pederson',
   'id': '13077770',
   'salary': 4000,
   'fpts': 21,


In [226]:
from datetime import timedelta, date

def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)
start_date = date(2017, 4, 2)
end_date = date(2017, 10, 2)
# start_date = date(2018, 3, 29)
# end_date = date(2018, 10, 1)
# start_date = date(2019, 3, 28)
# end_date = date(2019, 10, 1)

In [227]:
for single_date in daterange(start_date, end_date):

    resultsdb_date = single_date.strftime("%m/%d/%Y")
    date = single_date.strftime("%Y-%m-%d")
    # Get the slate id
    daily_slate = get_slates(resultsdb_date, SITE, sport="baseball")
    for slate in daily_slate:
        SLATEID = slate['_id']
        # skip slates where gameCount < 5
        if slate.get('gameCount', 0) < 5:
            continue
        # Get the names of the contests in the slate
        contests_in_slate = get_contests_from_slate(SLATEID)
        contest_names = [contest['name'] for contest in contests_in_slate]
        # Fetch daily ownerships
        daily_ownerships = get_contest_ownerships(SLATEID)
        # Parse data into a list to add to dataframe
        appended_player_data = []
        for player in daily_ownerships:
            player_data = daily_ownerships[player]
            player_row = [player_data.get('name', None), player_data.get('slatePosition', None),
                          player_data.get('salary', None), player_data.get('actualFpts', 0),
                          player_data.get('projectedFpts', 0), player_data.get('projectedOwnership', 0),
                          round(player_data.get('combinedOwnership', 0), 4)]
            player_contest_row = calculate_tiers(contests_in_slate)
            joined_list = player_row + player_contest_row + [date] + [SLATEID]
            appended_player_data.append(joined_list)
        # Create table structure from the data
        df = pd.DataFrame(appended_player_data, columns=['name',
         'slatePosition','salary', 'actualFpts', 'projectedFpts', 'projectedOwnership',
         'combinedOwnership', 'low_tier', 'mid_tier', 'high_tier', 'double_ups', 'date', 'slate_id'])
        df.to_csv('../data/resultsdb/TierContests/2017/{}-ownerships-{}.csv'.format(date, SLATEID))

In [218]:
def calculate_tiers(contests_in_slate):
    double_ups = filter((lambda x: "Double Up" in x['name']), contests_in_slate)
    double_ups = [contest['name'] for contest in double_ups]

    low_tier = [contest['name'] for contest in contests_in_slate if contest['entryFee'] <= 7]
    mid_tier = [contest['name'] for contest in contests_in_slate if (contest['entryFee'] > 7 and contest['entryFee'] < 25)]
    high_tier = [contest['name'] for contest in contests_in_slate if contest['entryFee'] >= 25]

    low_tier = [x for x in low_tier if x not in double_ups]
    mid_tier = [x for x in mid_tier if x not in double_ups]
    high_tier = [x for x in high_tier if x not in double_ups]
    
    low_tier_ownerships = [player_data.get(contest_name, 0) for contest_name in low_tier]
    mid_tier_ownerships = [player_data.get(contest_name, 0) for contest_name in mid_tier]
    high_tier_ownerships = [player_data.get(contest_name, 0) for contest_name in high_tier]
    double_ups_ownerships = [player_data.get(contest_name, 0) for contest_name in double_ups]
    
    if len(low_tier_ownerships) != 0:
        average_low_tier = round(sum(low_tier_ownerships)/len(low_tier_ownerships), 4)
    else:
        average_low_tier = 0
    if len(mid_tier_ownerships) != 0:
        average_mid_tier = round(sum(mid_tier_ownerships)/len(mid_tier_ownerships), 4)
    else:
        average_mid_tier = 0
    if len(high_tier_ownerships) != 0:
        average_high_tier = round(sum(high_tier_ownerships)/len(high_tier_ownerships), 4)
    else:
        average_high_tier = 0
    if len(double_ups_ownerships) != 0:
        average_double_ups = round(sum(double_ups_ownerships)/len(double_ups_ownerships), 4)
    else:
        average_double_ups = 0
        
    return [average_low_tier, average_mid_tier, average_high_tier, average_double_ups]